In [2]:
from torch.utils.data import DataLoader, Dataset
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoConfig, RobertaTokenizerFast
from torch.utils.data import TensorDataset, DataLoader
device = torch.device("cuda")


In [5]:
class BaseDataset(Dataset):
    def __init__(
        self,
        path_to_data="../data",
        model_name="distilroberta-base",
        ground_truth="risk_golden_truth_chunks.txt",
        processed_folder="chunked",
    ):
        self.labels = []
        self.ext = ".txt"

        self.path_to_data = path_to_data
        self.path_to_processed = os.path.join(self.path_to_data, processed_folder)
        self.path_labels = os.path.join(self.path_to_data, ground_truth)

        self.tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
        with open(self.path_labels, "r") as f:
            for line in f:
                subject, label = line.split()
                self.labels.append((subject, label))

    def __getitem__(self, idx):
        subject, label = self.labels[idx]
        subject_path = os.path.join(self.path_to_processed, subject) + self.ext
        with open(subject_path, "r") as f:
            text = " ".join(f.readlines())

        text = self.tokenizer.encode_plus(
            text,
            padding="max_len",
            max_len=512,
            truncation=True,
        )
        return {
            "ids": torch.tensor(text['input_ids'], dtype=torch.float),
            "mask": torch.tensor(text["attention_mask"], dtype=torch.float),
            "labels": torch.tensor(int(label), dtype=torch.float),
        }

    def __len__(self):
        return len(self.labels)


dl = DataLoader(BaseDataset(model_name=r"../models/roberta-tokenizer"), batch_size=4)

file ../models/roberta-tokenizer/config.json not found
file ../models/roberta-tokenizer/config.json not found


In [6]:
for x in dl:
    print(x)
    break